<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 50000 recenzji filmów oznaczonych sentymentem: pozytywny/negatywny. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [ ]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

!pip uninstall -y tensorflow
!pip install -q tensorflow==2.0.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [ ]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [ ]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [ ]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [ ]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [ ]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [ ]:
test_data.shape

(25000, 10000)

### <a name='a3'></a> 3. Budowa modelu bazowego

In [ ]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 216us/sample - loss: 0.5458 - accuracy: 0.7068 - binary_crossentropy: 0.5458 - val_loss: 0.3835 - val_accuracy: 0.8724 - val_binary_crossentropy: 0.3835
Epoch 2/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.2683 - accuracy: 0.9103 - binary_crossentropy: 0.2683 - val_loss: 0.2878 - val_accuracy: 0.8864 - val_binary_crossentropy: 0.2878
Epoch 3/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.1823 - accuracy: 0.9375 - binary_crossentropy: 0.1823 - val_loss: 0.2974 - val_accuracy: 0.8826 - val_binary_crossentropy: 0.2974
Epoch 4/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.1414 - accuracy: 0.9526 - binary_crossentropy: 0.1414 - val_loss: 0.3181 - val_accuracy: 0.8791 - val_binary_crossentropy: 0.3181
Epoch 5/20
25000/25000 [==============================] - 4s 141us/sample - loss: 0.1128 - acc

### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

In [ ]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 40004     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [ ]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 163us/sample - loss: 0.6270 - accuracy: 0.6866 - binary_crossentropy: 0.6270 - val_loss: 0.5127 - val_accuracy: 0.8469 - val_binary_crossentropy: 0.5127
Epoch 2/20
25000/25000 [==============================] - 3s 138us/sample - loss: 0.4015 - accuracy: 0.8791 - binary_crossentropy: 0.4015 - val_loss: 0.3628 - val_accuracy: 0.8756 - val_binary_crossentropy: 0.3628
Epoch 3/20
25000/25000 [==============================] - 3s 136us/sample - loss: 0.2876 - accuracy: 0.9090 - binary_crossentropy: 0.2876 - val_loss: 0.3102 - val_accuracy: 0.8848 - val_binary_crossentropy: 0.3102
Epoch 4/20
25000/25000 [==============================] - 3s 136us/sample - loss: 0.2335 - accuracy: 0.9238 - binary_crossentropy: 0.2335 - val_loss: 0.2897 - val_accuracy: 0.8874 - val_binary_crossentropy: 0.2897
Epoch 5/20
25000/25000 [==============================] - 3s 136us/sample - loss: 0.2001 - acc

### <a name='a5'></a> 4. Budowa 'większego' modelu

In [ ]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 14s 569us/sample - loss: 0.3545 - accuracy: 0.8424 - binary_crossentropy: 0.3545 - val_loss: 0.2978 - val_accuracy: 0.8783 - val_binary_crossentropy: 0.2978
Epoch 2/20
25000/25000 [==============================] - 14s 546us/sample - loss: 0.1400 - accuracy: 0.9486 - binary_crossentropy: 0.1400 - val_loss: 0.3267 - val_accuracy: 0.8751 - val_binary_crossentropy: 0.3267
Epoch 3/20
25000/25000 [==============================] - 14s 544us/sample - loss: 0.0442 - accuracy: 0.9862 - binary_crossentropy: 0.0442 - val_loss: 0.4309 - val_accuracy: 0.8660 - val_binary_crossentropy: 0.4309
Epoch 4/20
25000/25000 [==============================] - 14s 542us/sample - loss: 0.0057 - accuracy: 0.9993 - binary_crossentropy: 0.0057 - val_loss: 0.6070 - val_accuracy: 0.8684 - val_binary_crossentropy: 0.6070
Epoch 5/20
25000/25000 [==============================] - 14s 544us/sample - loss: 0.0013 

In [ ]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.545835,0.70684,0.545835,0.383504,0.87244,0.383504,0
1,0.268264,0.91028,0.268264,0.287847,0.88636,0.287847,1
2,0.182265,0.93748,0.182265,0.297360,0.88264,0.297360,2
3,0.141434,0.95264,0.141434,0.318061,0.87908,0.318061,3
4,0.112782,0.96352,0.112782,0.348570,0.87408,0.348570,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

### <a name='a7'></a> 6. Metody regularyzacji

In [ ]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 247us/sample - loss: 0.6659 - accuracy: 0.7930 - binary_crossentropy: 0.5133 - val_loss: 0.5164 - val_accuracy: 0.8678 - val_binary_crossentropy: 0.3760
Epoch 2/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.4263 - accuracy: 0.8990 - binary_crossentropy: 0.2894 - val_loss: 0.4301 - val_accuracy: 0.8856 - val_binary_crossentropy: 0.2985
Epoch 3/20
25000/25000 [==============================] - 4s 141us/sample - loss: 0.3523 - accuracy: 0.9217 - binary_crossentropy: 0.2253 - val_loss: 0.4065 - val_accuracy: 0.8878 - val_binary_crossentropy: 0.2849
Epoch 4/20
25000/25000 [==============================] - 4s 142us/sample - loss: 0.3143 - accuracy: 0.9334 - binary_crossentropy: 0.1967 - val_loss: 0.3959 - val_accuracy: 0.8871 - val_binary_crossentropy: 0.2828
Epoch 5/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.2904 - acc

In [ ]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [ ]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 180us/sample - loss: 0.6474 - accuracy: 0.6074 - binary_crossentropy: 0.6474 - val_loss: 0.5360 - val_accuracy: 0.8468 - val_binary_crossentropy: 0.5360
Epoch 2/20
25000/25000 [==============================] - 4s 147us/sample - loss: 0.5059 - accuracy: 0.7613 - binary_crossentropy: 0.5059 - val_loss: 0.3876 - val_accuracy: 0.8795 - val_binary_crossentropy: 0.3876
Epoch 3/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3983 - accuracy: 0.8305 - binary_crossentropy: 0.3983 - val_loss: 0.3133 - val_accuracy: 0.8870 - val_binary_crossentropy: 0.3133
Epoch 4/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3263 - accuracy: 0.8754 - binary_crossentropy: 0.3263 - val_loss: 0.2823 - val_accuracy: 0.8890 - val_binary_crossentropy: 0.2823
Epoch 5/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.2748 - acc

In [ ]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()